In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os

In [2]:
## Loading HAAR classifier
classifier = cv2.CascadeClassifier(r'..\haarcascade_frontalface_default.xml')

In [3]:
def extractface(input_image):
    ## Return cropped face
    ##gray = cv2.cvtColor(input_image,cv2.COLOR_BGR2GRAY)
    face = classifier.detectMultiScale(input_image,1.3,5)
    if face is ():
        return None
    for (x,y,w,h) in face:
        cv2.rectangle(input_image,(x,y),(x+w,y+h),(0,255,0),2)
        cropped = input_image[y:y+h,x:x+w]
    return cropped

In [4]:
cam = cv2.VideoCapture(1)
i=0
while True:
    ret,frame = cam.read()
 
    if extractface(frame) is not None:
        i +=1
        #face = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
        try:
            face = cv2.resize(extractface(frame),(300,300))

            try:
                file_path = r'..\\'+str(i)+'.jpeg'
                cv2.imwrite(file_path,face)
            except cv2.error as e:
                print('invalid image')
            cv2.putText(frame,'face',(100,100),cv2.FONT_HERSHEY_COMPLEX,1,(0,255,0),2)
            cv2.imshow('Face',frame)
        except cv2.error as e:
            print('invalid')
        
    #cv2.imshow('vide feed', frame)
    else:
        cv2.putText(frame,'No face',(100,100),cv2.FONT_HERSHEY_COMPLEX,1,(0,255,0),2)
        cv2.imshow('Face',frame)
        
    
    if cv2.waitKey(1)==13:
        break
cam.release()
cv2.destroyAllWindows()


invalid
invalid
invalid
invalid
invalid
invalid
invalid
invalid
invalid
invalid
invalid
invalid
invalid
invalid
invalid


In [5]:
## Training the machine learning model
data_path = r'..\\'
files = [ i for i in os.listdir(data_path) if os.path.isfile(os.path.join(data_path,i))]

data,label = [],[]

## Creating array for training data
for i,file in enumerate(files):
    image = data_path+files[i]
    images = cv2.imread(image,cv2.IMREAD_GRAYSCALE)
    data.append(images)  
    label.append(i)


## Array with data and labels
label = np.asarray(label,dtype = np.int32)

## Model
recog = cv2.face.LBPHFaceRecognizer_create()
## Model Training
recog.train(np.asarray(data),np.asarray(label))
print('model trained')

model trained


In [6]:
## Running recognition

def detect_face(input_image,size = 0.5):
    gray = cv2.cvtColor(input_image,cv2.COLOR_BGR2GRAY)
    face = classifier.detectMultiScale(gray,1.5,5)

    if face is ():
        return input_image,None
    
    for (x,y,w,h) in face:
        cv2.rectangle(input_image,(x,y),(x+w,y+h),(0,0,255),2)
        roi = input_image[y:y+h,x:x+w]
        roi = cv2.resize(roi,(200,200))
        
    return input_image,roi




In [7]:
camera = cv2.VideoCapture(1)

while True:
    ret,frame = camera.read()
    image,face = detect_face(frame)

    try:
        if face is not None:

            face = cv2.cvtColor(face,cv2.COLOR_BGR2GRAY)

            result = recog.predict(face)

            if result[1]<600:
                conf = int(100*(1-(result[1])/300))
                display = str(conf)+'%'
        
                cv2.putText(image,display,(100,100),cv2.FONT_HERSHEY_COMPLEX,1,(0,0,255),2)

            if conf>75:
                cv2.putText(image,'Person',(100,150),cv2.FONT_HERSHEY_COMPLEX,1,(0,255,0),2)
                cv2.imshow('Recognizer',image)
            else:
                cv2.putText(image,'Not Person',(100,200),cv2.FONT_HERSHEY_COMPLEX,1,(0,0,255),2)
                cv2.imshow('Recognizer',image)
        else :
            cv2.imshow('Recognizer',image)

    except cv2.error as e:
        cv2.putText(image,'No Face',(100,100),cv2.FONT_HERSHEY_COMPLEX,1,(0,0,255),2)
        cv2.imshow('Recognizer',image)
        

    if cv2.waitKey(1) == 13:
        break
camera.release()
cv2.destroyAllWindows()


In [42]:
camera.release()
